In [13]:
#imports
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [14]:
#fetch url
url = "https://www.mediaite.com/daily-ratings/cable-news-ratings-friday-june-9-msnbc-dominates-fox-news-and-cnn-in-total-day-and-prime-time/"
base = requests.get(url)
base

<Response [200]>

In [15]:
#convert to html text & test
base_html = base.text 
print(base_html)

<!DOCTYPE html><html lang="en"><head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5, minimum-scale=1.0">
<meta name="date" content="2023-06-12T16:32:41+00:00">
<!-- Open Graph -->
<!-- End Open Graph -->
<link rel="shortcut icon" href="https://www.mediaite.com/favicon.ico" /><link rel="apple-touch-icon" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png"><link rel="icon" sizes="192x192" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png">
		<!-- All in One SEO 4.4.0.1 - aioseo.com -->
		<title>Cable News Ratings Friday June 9: MSNBC, CNN &amp; Fox News</title>
		<meta name="description" content="MSNBC scored 1.46 million total day viewers and 179,000 demo viewers for first place. Fox News came in second with 1.28 million total day viewers and 136,000 demo viewers." />
		<meta name="robots" content="max-snippet:-1, max-image-preview:large, max-video-preview:-1" />
		<meta n

In [16]:
#make soup object
soup = BeautifulSoup(base_html)
type(soup)

bs4.BeautifulSoup

In [17]:
#selecting just the first of two tables on the page
table = soup.select("table")[0]

In [18]:
#creating header list
first_row = table.select("tbody tr")[0]
header_list = [ cell.text.rstrip("\n") for cell in first_row.select("td") ]
header_list

['ET', 'FNC', 'CNN', 'MSNBC', 'NEWSMAX', 'NEWSNATION']

In [19]:
#selecting non header rows of data from table 
rows_ratings = table.select("tr")[1:19]
len(rows_ratings)
print(rows_ratings)

[<tr>
<td data-sheets-value='{"1":2,"2":"6a"}'>6a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n827"}'>FOX AND FRIENDS:<br/>
827</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n326"}'>CNN THIS MORNING:<br/>
326</td>
<td data-sheets-value='{"1":2,"2":"MORNING JOE:\n1240"}'>MORNING JOE:<br/>
1240</td>
<td data-sheets-value='{"1":2,"2":"CONVERSATIONS WITH NANCY:\n36"}'>CONVERSATIONS WITH NANCY:<br/>
36</td>
<td data-sheets-value='{"1":2,"2":"EARLY MORNING:\n13"}'>EARLY MORNING:<br/>
13</td>
</tr>, <tr>
<td data-sheets-value='{"1":2,"2":"7a"}'>7a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n1089"}'>FOX AND FRIENDS:<br/>
1089</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n384"}'>CNN THIS MORNING:<br/>
384</td>
<td data-sheets-value='{"1":2,"2":"--"}'>—</td>
<td data-sheets-value='{"1":2,"2":"WAKE UP AMERICA:\n181"}'>WAKE UP AMERICA:<br/>
181</td>
<td data-sheets-value='{"1":2,"2":"MORNING IN AMERICA:\n37"}'>MORNING IN AMERICA:<br/>
37</td>
</t

In [20]:
#creating a list of cells from the table
rating_entries = []
for row in rows_ratings:
    row_cells = []
    for cell in row.select("td"):
        row_cells.append(cell.text.rstrip("\n"))
    rating_entries.append(row_cells)
rating_entries[:3]

[['6a',
  'FOX AND FRIENDS:\n827',
  'CNN THIS MORNING:\n326',
  'MORNING JOE:\n1240',
  'CONVERSATIONS WITH NANCY:\n36',
  'EARLY MORNING:\n13'],
 ['7a',
  'FOX AND FRIENDS:\n1089',
  'CNN THIS MORNING:\n384',
  '—',
  'WAKE UP AMERICA:\n181',
  'MORNING IN AMERICA:\n37'],
 ['8a', 'FOX AND FRIENDS:\n1205', 'CNN THIS MORNING:\n485', '—', '—', '—']]

In [21]:
cable9_df = pd.DataFrame(rating_entries, columns=header_list)
cable9_df.head()

,ET,FNC,CNN,MSNBC,NEWSMAX,NEWSNATION
0,6a,FOX AND FRIENDS:\n827,CNN THIS MORNING:\n326,MORNING JOE:\n1240,CONVERSATIONS WITH NANCY:\n36,EARLY MORNING:\n13
1,7a,FOX AND FRIENDS:\n1089,CNN THIS MORNING:\n384,—,WAKE UP AMERICA:\n181,MORNING IN AMERICA:\n37
2,8a,FOX AND FRIENDS:\n1205,CNN THIS MORNING:\n485,—,—,—
3,9a,AMERICAS NEWSROOM:\n1325,CNN NEWS CENTRAL:\n619,MORNING JOE:\n1234,NATIONAL REPORT:\n236,—
4,10a,AMERICAS NEWSROOM:\n1288,CNN NEWS CENTRAL:\n643,ANA CABRERA REPORTS:\n962,—,NWSN LIVE W/ MARNI HUGHES:\n28


In [22]:
#dropping everything but fox to create fox column only df
fox9 = cable9_df.drop(columns=['CNN', 'MSNBC','NEWSMAX',"NEWSNATION","ET"])
fox9

,FNC
0,FOX AND FRIENDS:\n827
1,FOX AND FRIENDS:\n1089
2,FOX AND FRIENDS:\n1205
3,AMERICAS NEWSROOM:\n1325
4,AMERICAS NEWSROOM:\n1288
5,"FAULKNER FOCUS, THE:\n1241"
6,OUTNUMBERED:\n1285
7,AMERICA REPORTS:\n1195
8,AMERICA REPORTS:\n1201
9,"STORY, THE:\n1250"


In [23]:
#splitting the string and numbers in the FNC column, while dropping the :\n . And then dropping the original FNC Column
fox9[['Show', '6/9']] = fox9['FNC'].str.split(':\n', expand=True)
fox9 = fox9.drop(columns=['FNC'])
fox9

,Show,6/9
0,FOX AND FRIENDS,827
1,FOX AND FRIENDS,1089
2,FOX AND FRIENDS,1205
3,AMERICAS NEWSROOM,1325
4,AMERICAS NEWSROOM,1288
5,"FAULKNER FOCUS, THE",1241
6,OUTNUMBERED,1285
7,AMERICA REPORTS,1195
8,AMERICA REPORTS,1201
9,"STORY, THE",1250


In [24]:
fox9.to_csv("fox9.csv")